In [2]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2
from src.data import make_dataset
from dotenv import find_dotenv, load_dotenv
import os
from stravalib.client import Client
import time
import csv

In [53]:
"""
To get data we have to deal with OAuth
1. Run this cell
2. Click it's output
3. Hit authorize
4. Copy the "code" parameter in the redirect and paste it in the cell below
5. Run the cell below
6. To refresh, run the cell below that
"""
client = Client()
authorize_url = client.authorization_url(client_id=22120, redirect_uri='http://localhost:8282/authorized')
# Have the user click the authorization URL, a 'code' param will be added to the redirect_uri
# .....
print(authorize_url)


https://www.strava.com/oauth/authorize?client_id=22120&redirect_uri=http%3A%2F%2Flocalhost%3A8282%2Fauthorized&approval_prompt=auto&response_type=code&scope=read%2Cactivity%3Aread


In [56]:
# Extract the code from your webapp response
code = "4f9497b3487b523145a7d9664a7a9e8b50c8c1a0" # or whatever your framework does
token_response = client.exchange_code_for_token(client_id=os.getenv("client_id"), client_secret=os.getenv("client_secret"), code=code)
access_token = token_response['access_token']
refresh_token = token_response['refresh_token']
expires_at = token_response['expires_at']

# Now store that short-lived access token somewhere (a database?)
client.access_token = access_token
# You must also store the refresh token to be used later on to obtain another valid access token 
# in case the current is already expired
client.refresh_token = refresh_token

# An access_token is only valid for 6 hours, store expires_at somewhere and
# check it before making an API call.
client.token_expires_at = expires_at
 
athlete = client.get_athlete()
print("For {id}, I now have an access token {token}".format(id=athlete.id, token=access_token))



For 25983534, I now have an access token 963731c1306d483dd0a7ade62496aeb7b3e3868c


In [ ]:
# ... time passes ...
if time.time() > client.token_expires_at:
    refresh_response = client.refresh_access_token(client_id=os.getenv("client_id"), client_secret=os.getenv("client_secret"),
        refresh_token=client.refresh_token)
    access_token = refresh_response['access_token']
    refresh_token = refresh_response['refresh_token']
    expires_at = refresh_response['expires_at']

In [82]:
# Get the data
activities = client.get_activities()
types = ['time', 'latlng', 'altitude', 'heartrate', 'temp']
headers_written = False
#stream_types = ['time', 'latlng', 'altitude', 'heartrate', 'temp']
stream_types = ['heartrate']
w = None
with open(os.path.join("/home/greg/repos/commute_analysis", "data", "raw", 'raw_strava_data.csv'), 'w') as f:
    for activity in activities:
        streams = client.get_activity_streams(activity.id, types=stream_types, resolution='medium')
        temp = activity.to_dict()
        for k in types:
            if streams and k in streams:
                temp[k] = streams[k].data
            elif k not in temp:
                temp[k] = None
        if not headers_written:
            w = csv.DictWriter(f, temp.keys())
            w.writeheader()
            headers_written = True
        w.writerow(temp)


In [89]:
make_dataset.convert_strava_data()

/home/greg/.virtualenvs/commute_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->['external_id', 'athlete', 'name', 'type', 'start_date', 'timezone', 'start_latlng', 'end_latlng', 'map', 'device_watts', 'from_accepted_tag', 'heartrate']]

  """Entry point for launching an IPython kernel.


In [8]:
BASE_PATH = ".."
import pandas as pd

In [27]:
make_dataset.convert_data()